In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import random

In [2]:
df = pd.read_excel('./data/环球影城等候时间数据-xr+lk+gs v2.xlsx', sheet_name='Sheet1')

In [24]:
df['等候时间'].max(), df['等候时间'].min()

(90, 5)

In [3]:
week = pd.read_excel('./data/环球影城平均等候时间-lk+xr+gs.xlsx', sheet_name='Sheet1')

In [4]:
df.head()

,项目,时间,等候时间
0,变形金刚,2021-10-19 09:00:00,7
1,变形金刚,2021-10-19 09:15:00,31
2,变形金刚,2021-10-19 09:30:00,40
3,变形金刚,2021-10-19 09:45:00,41
4,变形金刚,2021-10-19 10:00:00,50


In [5]:
week.head()

,项目,周日,周六,周五,周四,周三,周二,周一
0,变形金刚,58,89,30,49,39,41,61
1,哈利波特与禁忌之旅,55,112,26,49,41,48,57
2,神偷奶爸,20,48,11,19,15,16,23
3,侏罗纪世界,55,102,26,49,38,42,55
4,霸天虎过山车,38,71,28,48,36,44,61


In [6]:
df.loc[df['等候时间'].isnull(), '等候时间'] = '/'

In [7]:
df = df.loc[df['等候时间'] != '/']

In [8]:
df

,项目,时间,等候时间
0,变形金刚,2021-10-19 09:00:00,7
1,变形金刚,2021-10-19 09:15:00,31
2,变形金刚,2021-10-19 09:30:00,40
3,变形金刚,2021-10-19 09:45:00,41
4,变形金刚,2021-10-19 10:00:00,50
...,...,...,...
828,鹰马飞行,2021-10-19 20:00:00,15
829,鹰马飞行,2021-10-19 20:15:00,12
830,鹰马飞行,2021-10-19 20:30:00,10
831,鹰马飞行,2021-10-19 20:45:00,10


In [9]:
# 10-19是周二
for weekday in ['周日', '周六', '周五', '周四', '周三', '周一']:
    df[f'{weekday}_ratio'] = 1
    for item in week['项目'].unique():
        r = float(week.loc[(week['项目'] == item), weekday]  / week.loc[(week['项目'] == item), '周二'].values)
        df.loc[df['项目'] == item, f'{weekday}_ratio'] = df.loc[df['项目'] == item, f'{weekday}_ratio'] * r
        df[f'{weekday}_等候时间'] = df[f'{weekday}_ratio'] * df['等候时间']
        df[f'{weekday}_等候时间'] = df[f'{weekday}_等候时间'].apply(lambda x: np.round(x, 0))

In [10]:
df.head(100)

,项目,时间,等候时间,周日_ratio,周日_等候时间,周六_ratio,周六_等候时间,周五_ratio,周五_等候时间,周四_ratio,周四_等候时间,周三_ratio,周三_等候时间,周一_ratio,周一_等候时间
0,变形金刚,2021-10-19 09:00:00,7,1.414634,10.0,2.170732,15.0,0.731707,5.0,1.195122,8.0,0.951220,7.0,1.487805,10.0
1,变形金刚,2021-10-19 09:15:00,31,1.414634,44.0,2.170732,67.0,0.731707,23.0,1.195122,37.0,0.951220,29.0,1.487805,46.0
2,变形金刚,2021-10-19 09:30:00,40,1.414634,57.0,2.170732,87.0,0.731707,29.0,1.195122,48.0,0.951220,38.0,1.487805,60.0
3,变形金刚,2021-10-19 09:45:00,41,1.414634,58.0,2.170732,89.0,0.731707,30.0,1.195122,49.0,0.951220,39.0,1.487805,61.0
4,变形金刚,2021-10-19 10:00:00,50,1.414634,71.0,2.170732,109.0,0.731707,37.0,1.195122,60.0,0.951220,48.0,1.487805,74.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,哈利波特与禁忌之旅,2021-10-19 20:45:00,15,1.145833,17.0,2.333333,35.0,0.541667,8.0,1.020833,15.0,0.854167,13.0,1.187500,18.0
98,神偷奶爸,2021-10-19 09:00:00,10,1.250000,12.0,3.000000,30.0,0.687500,7.0,1.187500,12.0,0.937500,9.0,1.437500,14.0
99,神偷奶爸,2021-10-19 09:15:00,10,1.250000,12.0,3.000000,30.0,0.687500,7.0,1.187500,12.0,0.937500,9.0,1.437500,14.0
100,神偷奶爸,2021-10-19 09:30:00,10,1.250000,12.0,3.000000,30.0,0.687500,7.0,1.187500,12.0,0.937500,9.0,1.437500,14.0


In [11]:
data = df[['项目', '时间', '等候时间']].copy()
week2day = {'周一': '10-18', 
            '周日': '10-17', 
            '周六': '10-16', 
            '周五': '10-15', 
            '周四': '10-14', 
            '周三': '10-13', 
           }

for i, cur_day in enumerate(week2day.keys()):
    temp = df[['项目', '时间', f'{cur_day}_等候时间']].copy()
    temp['时间'] = temp['时间'].apply(lambda x: str(x).replace('10-19', week2day[cur_day]))
    temp.rename({f'{cur_day}_等候时间': '等候时间'}, axis=1, inplace=True)
    data = data.append(temp)

In [12]:
data

,项目,时间,等候时间
0,变形金刚,2021-10-19 09:00:00,7
1,变形金刚,2021-10-19 09:15:00,31
2,变形金刚,2021-10-19 09:30:00,40
3,变形金刚,2021-10-19 09:45:00,41
4,变形金刚,2021-10-19 10:00:00,50
...,...,...,...
828,鹰马飞行,2021-10-13 20:00:00,15.0
829,鹰马飞行,2021-10-13 20:15:00,12.0
830,鹰马飞行,2021-10-13 20:30:00,10.0
831,鹰马飞行,2021-10-13 20:45:00,10.0


In [13]:
col = '时间'
prefix = ''
data[col] = pd.to_datetime(data[col])
data[prefix + 'year'] = data[col].dt.year
data[prefix + 'month'] = data[col].dt.month
data[prefix + 'day'] = data[col].dt.day

In [14]:
data

,项目,时间,等候时间,year,month,day
0,变形金刚,2021-10-19 09:00:00,7,2021,10,19
1,变形金刚,2021-10-19 09:15:00,31,2021,10,19
2,变形金刚,2021-10-19 09:30:00,40,2021,10,19
3,变形金刚,2021-10-19 09:45:00,41,2021,10,19
4,变形金刚,2021-10-19 10:00:00,50,2021,10,19
...,...,...,...,...,...,...
828,鹰马飞行,2021-10-13 20:00:00,15.0,2021,10,13
829,鹰马飞行,2021-10-13 20:15:00,12.0,2021,10,13
830,鹰马飞行,2021-10-13 20:30:00,10.0,2021,10,13
831,鹰马飞行,2021-10-13 20:45:00,10.0,2021,10,13


In [15]:
for cur_day in range(19, 7, -1):
    temp = data.loc[(data['month'] == 10) & (data['day'] == cur_day)].copy()
    
    randomlist = random.sample(range(1, 2000), 815)
    randomlist = [(x - 1000) / 10000 + 1 for x in randomlist]

    temp['等候时间'] = temp['等候时间'] * randomlist
    temp['等候时间'] = temp['等候时间'].apply(lambda x: round(x, 0))
    print(cur_day)
    print(f'10-{cur_day-7}')
    if cur_day >= 10:
        temp['时间'] = temp['时间'].apply(lambda x: str(x).replace(f'10-{cur_day}', f'10-{cur_day-7}'))
    else:
        temp['时间'] = temp['时间'].apply(lambda x: str(x).replace(f'10-0{cur_day}', f'10-0{cur_day-7}'))
    data = data.append(temp)
    
    col = '时间'
    prefix = ''
    data[col] = pd.to_datetime(data[col])
    data[prefix + 'year'] = data[col].dt.year
    data[prefix + 'month'] = data[col].dt.month
    data[prefix + 'day'] = data[col].dt.day

19
10-12
18
10-11
17
10-10
16
10-9
15
10-8
14
10-7
13
10-6
12
10-5
11
10-4
10
10-3
9
10-2
8
10-1


In [18]:
randomlist = random.sample(range(1, 15489), 15485)
randomlist = [(x - 15485//2) / 15485*50 + 1 for x in randomlist]

In [25]:
max(randomlist), min(randomlist)

(26.011301259283176, -23.995156603164354)

In [27]:
data['等候时间'] = data['等候时间'] + randomlist
data['等候时间'] = data['等候时间'].apply(lambda x: round(x, 0))


In [30]:
data['等候时间'].max(), data['等候时间'].min()

(200.0, -21.0)

In [32]:
data['等候时间'] = np.clip(data['等候时间'], 5, 120)

In [33]:
data

,项目,时间,等候时间,year,month,day
0,变形金刚,2021-10-19 09:00:00,17.0,2021,10,19
1,变形金刚,2021-10-19 09:15:00,12.0,2021,10,19
2,变形金刚,2021-10-19 09:30:00,42.0,2021,10,19
3,变形金刚,2021-10-19 09:45:00,50.0,2021,10,19
4,变形金刚,2021-10-19 10:00:00,61.0,2021,10,19
...,...,...,...,...,...,...
828,鹰马飞行,2021-10-01 20:00:00,37.0,2021,10,1
829,鹰马飞行,2021-10-01 20:15:00,5.0,2021,10,1
830,鹰马飞行,2021-10-01 20:30:00,10.0,2021,10,1
831,鹰马飞行,2021-10-01 20:45:00,7.0,2021,10,1


In [34]:
del_cols = ['year', 'month', 'day']
data.drop(del_cols, axis=1, inplace=True)

In [35]:
data.to_csv("./data/generate_data.csv", index = False)